# PID interconnects PCB

Import packages:

In [1]:
import pyvisa
import time
import matplotlib.pyplot as plt
import matplotlib.patches as plt_patches
from tqdm import tqdm_notebook
import numpy as np
import h5py

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Open the data file with read/write append access:

In [ ]:
f = h5py.File('data2.hdf5', 'a')

Connect to the serial devices:

In [2]:
rm = pyvisa.ResourceManager()

In [3]:
rm.list_resources()

('ASRL1::INSTR', 'ASRL13::INSTR', 'ASRL15::INSTR')

In [5]:
arduino = rm.open_resource('ASRL13::INSTR')

In [370]:
arduino.close()

### Arduino for serial communications

Define functions for communicating with the FPGA over Arduino:

In [364]:
def read_FPGA(cmd, num_bytes):
    # send read command
    arduino.write_raw(bytes(cmd, encoding="ascii"))
    time.sleep(0.01)
    
    # collect data
    received_data = []
    for i in range(num_bytes):
        received_data.append( int(arduino.read()[:-2], 16) )
        
    return received_data

def set_FPGA(cmd, data_list):
    # send write command
    arduino.write_raw(bytes(cmd, encoding="ascii"))
    time.sleep(0.01)
    
    # send data
    for val in data_list:
        arduino.write_raw(bytes(bytearray([val])))
        time.sleep(0.01)    

Define convenience functions for accessing individual PID parameters:

In [365]:
def set_PID_param1(param, val):
    # parameter sanity check
    if not param in ["T", "F", "N"]:
        print("ERROR: Invalid parameter.")
        return
    
    # value sanity check
    if val > 3:
        print("ERROR: Invalid value.")
        return
    
    # send 2-byte value
    set_FPGA(param, [int(val)])
    
def set_PID_param2(param, val):
    # parameter sanity check
    if not param in ["P", "I", "S", "L"]:
        print("ERROR: Invalid parameter.")
        return
    
    # value sanity check
    if val > 0xffff:
        print("ERROR: Invalid value.")
        return
    
    # send 2-byte value
    set_FPGA(param, [int(val%256), int(val/256)])
    
def read_PID_param(param):
    # parameter sanity check
    if param in ["t", "f", "n"]:
        num_bytes = 1
    elif param in ["p", "i", "d", "s", "l"]:
        num_bytes = 2
    elif param in ["a", "e", "o"]:
        num_bytes = 4
    else:
        print("ERROR: Invalid parameter.")
        return
    
    # read raw data
    raw_result = read_FPGA(param, num_bytes)
    
    # convert bytes to integer
    if param in ["e", "a"]:
        int_result = int.from_bytes(raw_result, byteorder='big', signed=True)
    else:
        int_result = int.from_bytes(raw_result, byteorder='big', signed=False)
    
    return int_result

Define functions to read and write to the AD9910 registers:

In [366]:
def read_one_register(reg, printing=True):
    # send the read command
    arduino.write_raw(b"1")
    arduino.write_raw(bytes([reg]))
    
    # get Arduino's return text
    return_text = arduino.read(), arduino.read()
    
    if printing:
        for t in return_text:
            print(t, end='')
    
    # convert to list of integers
    return [int(x,2) for x in return_text[1].strip()[:-1].split(",")]

def read_registers():
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')
        
def reset(arduino):
    print(arduino.query('0'))
    
def write_register(reg, data_list, printing=False):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    return_text = arduino.read(), arduino.read(), arduino.read(), arduino.read(),  arduino.read(),  arduino.read()
    if printing:
        for t in return_text:
            print(t, end='')

Define functions to calculate the frequency tuning word, and the amplitude scale factor:

In [367]:
def FTW(Fsysclk, Fout):
    # calculate the exact FTW
    FTW = 2**32 * (Fout / Fsysclk)
    
    # round to the nearest integer
    FTW_int = int(FTW)
    
    # convert to bytes
    FTW_bytes = FTW_int.to_bytes(4, byteorder='big')
    
    # convert to list of integers
    return [int(x) for x in FTW_bytes]

def ASF(ASF_perc):
    # convert from percentage to integer
    ASF_perc_int = int(ASF_perc/100 * 2**14-1)
    
    # convert to bytes
    FTW_bytes = ASF_perc_int.to_bytes(2, byteorder='big')
    
    # convert to list of integers
    return [int(x) for x in FTW_bytes]

Define functions to write (equal) amplitude & frequency values to all profiles:

In [368]:
def set_freq_amp(clock_freq, out_freq, amplitude_percentage=100):
    for reg in range(0x0E, 0x15 + 1):
        write_register(
            reg,
            ASF(amplitude_percentage) + [0b00000000, 0b00000000] + FTW(clock_freq, out_freq),
            printing=False
          )

Define functions to enable/disable certain features of the AD9910:

In [369]:
def enable_parallel_port(enable = False):
    # read register 0x01
    val = read_one_register(0x01, printing=False)
    
    # flip the "Parallel data port enable" bit
    if enable:
        set_PID_param1("T", 1) # set TxENABLE = 1
        val[3] |= 0b00010000
    else:
        set_PID_param1("T", 1) # set TxENABLE = 0
        val[3] &= 0b11101111
    
    # write the new register values
    write_register(0x01, val)
    
def parallel_mode(mode):
    if mode == "AM": # amplitude modulation
        set_PID_param1("F", 0b00)
    elif mode == "PM": # phase modulation
        set_PID_param1("F", 0b01)
    elif mode == "FM": # frequency modulation
        set_PID_param1("F", 0b10)
    elif mode == "AP": # amplitude & phase modulation
        set_PID_param1("F", 0b11)
    else:
        print("ERROR: Invalid mode.")
        return
        
def enable_ASF(enable = False):
    # read register 0x01
    val = read_one_register(0x01, printing=False)
    
    # flip the "Enable amplitude scale from single ton profiles" bit
    if enable:
        val[0] |= 0b00000001
    else:
        val[0] &= 0b11111110
    
    # write the new register values
    write_register(0x01, val)
        
def enable_refclk_divider(enable = False):
    # read register 0x02
    val = read_one_register(0x02, printing=False)
    
    # flip the "REFCLK input divider bypass" bit
    if enable:
        val[2] &= 0b01111111
    else:
        val[2] |= 0b10000000
    
    # write the new register values
    write_register(0x02, val)

### Test 1: PID control of AD9910 output power

Consider using the FPGA PID controller to control the RF power ouput from the AD9910 DDS IC, as measured by the Mini-Circuits ZX47-50-S+ power detector:

![](AD9910.png)

Set the AD9910 for 85 MHz output from a 384 MHz clock (internal clock divider disabled), with parallel port enabled and set for amplitude modulation:

In [357]:
set_freq_amp(clock_freq=384e6, out_freq=85e6)
enable_refclk_divider(False)
enable_parallel_port(True)
parallel_mode("AM")

In [362]:
set_PID_param1("N", 1) # invert output
set_PID_param2("P", 0) # proportional constant
set_PID_param2("I", 4) # integral constant
set_PID_param2("S", 2.1e4 ) # setpoint
set_PID_param2("L", 0xffff ) # output limit

In [ ]:
read_PID_param("L")

In [350]:
print(read_PID_param("s"))
print(read_PID_param("d"))
print(read_PID_param("e"))

20000
18152
-3434
